In [14]:
from langchain_google_genai import ChatGoogleGenerativeAI
from api import GEMINI_API_KEY
llm = ChatGoogleGenerativeAI(model = "gemini-2.0-flash" , api_key = GEMINI_API_KEY)

In [1]:
from langchain_community.document_loaders import UnstructuredURLLoader


In [2]:
urls = ['https://www.victoriaonmove.com.au/local-removalists.html' , 'https://www.victoriaonmove.com.au/index.html' , 'https://www.victoriaonmove.com.au/contact.html']
loader = UnstructuredURLLoader(urls=urls)
data = loader.load()

In [3]:
data

[Document(metadata={'source': 'https://www.victoriaonmove.com.au/local-removalists.html'}, page_content='Loading...\n\nLOCAL REMOVALS\n\nYour trusted partner in seamless moving and packing solutions!\n\nGoogle Rating\n\n5 stars, 111 reviews\n\nRequst A call for You:\n\nLocal removal services via "Victoria on move"\n\nVictoria on Move is your trusted local moving company in Melbourne, specializing in seamless relocation services. As experienced furniture movers and relocation experts, we provide top-notch packing and moving services tailored to your needs. Whether you\'re moving across town or relocating interstate, our professional movers ensure a stress-free experience. Count on Victoria on Move for reliable removal services, making us the preferred choice among local movers in Melbourne. Discover why we\'re recognized for our commitment to quality and customer satisfaction.\n\nApartment Moving\n\nEfficient and careful relocation services tailored for apartments of all sizes, ensuring

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000)
docs = text_splitter.split_documents(data)


print(f'Total number of documents : {len(docs)}')

Total number of documents : 11


In [5]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

/tmp/ipykernel_8135/3077754740.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
/tmp/ipykernel_8135/3077754740.py:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
/home/haseeb/WorkFolder/GenAI/DSwithBappy/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .auton

In [6]:
from langchain_chroma import Chroma
vectorstore = Chroma.from_documents(documents= docs , embedding=embeddings)

In [7]:
retriver = vectorstore.as_retriever(search_type = 'similarity'  , search_kwargs = {'k' : 3})

In [8]:
res = retriver.invoke("what kind of service they provide ?")

In [9]:
len(res)

3

In [10]:
res

[Document(id='defa821c-4309-432b-8987-5ba452219aa2', metadata={'source': 'https://www.victoriaonmove.com.au/local-removalists.html'}, page_content='Apartment Moving\n\nEfficient and careful relocation services tailored for apartments of all sizes, ensuring smooth transitions to your new home.\n\nVilla Moving\n\nComprehensive moving solutions for large residences and villas, handling valuable possessions with utmost care and precision.\n\nHousehold Moving\n\nFull-service moving options for households, including packing, loading, transportation, and unpacking services to simplify your move.\n\nOffice Moving\n\nSpecialized expertise in office relocations, minimizing downtime and ensuring your business operations continue seamlessly.\n\nFurniture Moving\n\nExperienced in handling furniture of all sizes and types, ensuring safe transport and setup in your new location.\n\nPacking and Unpacking Services\n\nOptional packing and unpacking services available to save you time and effort, using h

In [11]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate


system_prompt = (
    "You are helful assistant for question answering task."
    "Use the following pieces of retrived context to answer "
    "the question. if you don't know the answer say that you"
    "don't know. Use three sentences maximum and keep the "
    "answer concise"
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system" , system_prompt),
        ("human" , "{input}"),
    ]
)

In [15]:
questionAnswerChain = create_stuff_documents_chain(llm , prompt)
rag_chain = create_retrieval_chain(retriver , questionAnswerChain)

In [16]:
response = rag_chain.invoke({"input" : "What kind of services they provide ? "})
print(response['answer'])

They provide apartment, villa, household, and office moving services. They also specialize in furniture moving and offer optional packing and unpacking services. Customized moving plans are available to fit specific requirements.
